<a href="https://colab.research.google.com/github/bpawnzZ/colabNotebooks/blob/main/stable/rev_animated_v1_1_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/Google-Drive-Logo.png" height="50" alt="Google-Drive-Logo"/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>

from pathlib import Path
from contextlib import suppress
from sys import exit as end
from google.colab import output
from google.colab.drive import mount, flush_and_unmount as unmount

MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
FORCE_REMOUNT = True #@param {type:"boolean"}
CUSTOM_MOUNT_POINT = False #@param {type:"boolean"}
DEBUG_MODE = False #@param {type:"boolean"}

if CUSTOM_MOUNT_POINT:
  Custom_Mount_Path = input('Please enter Custom Mount Path:\n')
  if not Custom_Mount_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Mount_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Mount_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
else:
  Custom_Mount_Path = '/content/drive'

mount._DEBUG = DEBUG_MODE

if MODE == "MOUNT":
  print("Please Wait! Mounting...")
  mount(Custom_Mount_Path, force_remount= FORCE_REMOUNT)
  if not DEBUG_MODE:
    output.clear()
  print(f'Mounted at {Custom_Mount_Path}')
elif MODE == 'UNMOUNT':
  with suppress(ValueError):
    print('Please Wait! UNMOUNT in progress...')
    unmount()
  get_ipython().system_raw('rm -rf /root/.config/Google/DriveFS')

In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/Setup_img.png" height="50" alt="Setup-Logo"/></center>
#@markdown <center><h3><b>Setup & Configure Environment</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath
from time import sleep
from requests import get as get_url
from urllib.parse import urlparse as validate_url
from google.colab import output
from json import dumps as dump_json

current_version = "v5.1.0"
do_normal_setup = "1"

# Check for latest version
try:
    json_data = get_url("https://assets.thecaduceus.eu.org/MCT/json/mct.json").json()
    print("Connected with CDN.")
    if current_version == json_data["version"]:
        print(f'Great Job! you are using latest version: {json_data["version"]}')
    else:
        print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]}, use latest notebook available here:\n{json_data["notebookLink"]}')
        sleep(10)
except Exception as error_code:
    end(f'Failed to connect with CDN!\nTraceback:\n{error_code}')

# Save json data locally
with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))

# Flavor Flags
Stable = ""
Beta = "-s beta"

# User Input
INSTALL_RCLONE_FLAVOR = Stable #@param ["Stable", "Beta"] {type:"raw"}
FORCE_SETUP = True #@param {type:"boolean"}
CUSTOM_SH_FILE = False #@param {type:"boolean"}

if CUSTOM_SH_FILE:

    Custom_sh_Source = input("Provide the Custom Install.sh file source, it can be URL or File Path:\n")
    use_sudo = input("Do you want to execute the .sh file as sudo? 1 = Yes and 0 = No:\n")

    if use_sudo not in ['0','1']:
        end("Unable to check for latest version!")
    elif not Custom_sh_Source:
        end("Custom sh file source can't be blank!")
    elif all([validate_url(Custom_sh_Source).scheme, validate_url(Custom_sh_Source).netloc]):
        try:
            get_url(Custom_sh_Source)
            if use_sudo == '1':
                !curl {Custom_sh_Source} | sudo bash
                print('Execution completed as sudo!')
            elif use_sudo == '0':
                !curl {Custom_sh_Source} | bash
                print('Execution completed!')
        except ConnectionError:
            end("Unable to connect with given sh file URL.")
        except Exception as error_code:
            end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")
    elif not Path(Custom_sh_Source).exists():
        end("Provided Custom sh file path not exist!")
    elif not Path(Custom_sh_Source).is_file():
        end("Provided Custom sh file path should of a particular file not directory/folder.")
    elif not PurePath(Custom_sh_Source).suffix == '.sh':
        end("File format should be .sh")
    else:
        try:
            if use_sudo == '1':
                !sudo bash "$Custom_sh_Source"
                print("Execution completed as sudo.")
            else:
                !bash "$Custom_sh_Source"
                print("Execution complete.")
        except Exception as error_code:
            end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")


    # Ask for Normal Setup
    do_normal_setup = input('Do you want to run Normal Setup also? 1 = Yes and 0 = No:\n')

    if do_normal_setup not in ['0','1']:
        end("Input Error: Value should be 1 or 0")

# Force Setup
if FORCE_SETUP:
    if Path("/content/setup.sh").exists():
        !rm {"/content/setup.sh"}
    print("Starting Force Setup...")
    sleep(4)
    !curl "https://rclone.org/install.sh" | sudo bash {INSTALL_RCLONE_FLAVOR}

if do_normal_setup == '1' and not Path("/content/setup.sh").exists():
    try:
        get_url(json_data["setupFile"])
        !curl {json_data["setupFile"]} | sudo bash {INSTALL_RCLONE_FLAVOR}
        !curl {json_data["setupFile"]} > "/content/setup.sh"
        output.clear()
        print("Setup completed!")
    except ConnectionError:
        print('Unable to get setup file!')
    except Exception as error_code:
        print(f'Setup instructions seems to be having mistake(s)!\nTraceback:\n{error_code}')
elif do_normal_setup == '1' and Path("/content/setup.sh").exists():
    print("Setup file exist and configured already!")

In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/rclone_logo.png" height="50" alt="Rclone-Logo"/></center>
#@markdown <center><h3><b>Fetch Previous MCT Config from My Drive</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')

#@markdown ---
CUSTOM_IMPORT_PATH = False #@param {type:"boolean"}
CUSTOM_EXPORT_PATH = False #@param {type:"boolean"}

# Import Configuration(s)

if CUSTOM_IMPORT_PATH:
  Custom_Path = input('Enter the directory path from where configuration file(s) needs to be imported:\n')
  # Check Path
  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
  elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
    !rclone mkdir "$DRIVE_CONFIG_DIR"
    print('Created: MCT-Config [folder]')

  Custom_Path = PurePath(Custom_Path) # Remove any extra '/'

  # Paths
  IMPORT_TO = DRIVE_CONFIG_DIR
  IMPORT_RCLONE_CONF = f"{Custom_Path}/rclone.conf"
  IMPORT_ACCOUNTS_ZIP = f"{Custom_Path}/accounts.zip"
  IMPORT_ACCOUNTS_FOLDER = f"{Custom_Path}/accounts/"
  ZIP_ACCOUNTS_FOLDER = f"{CMP}/MyDrive/MCT-Config/accounts" # accounts is name of zip, zip file will be saved in MCT-Config
  IMPORT_MEGA_JSON = f"{Custom_Path}/mega.json"
  IMPORT_GOFILE_JSON = f"{Custom_Path}/gofile.json"
  IMPORT_TELEGRAM_JSON = f"{Custom_Path}/telegram.json"
  IMPORT_SESSION_FILES = list(Path(Custom_Path).glob("*.session"))
  # Check 'rclone.conf' file

  if Path(IMPORT_RCLONE_CONF).exists():

    is_import = input('Found: rclone.conf [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_RCLONE_CONF" "$IMPORT_TO"
      print('Successfully imported "rclone.conf" file! (1)')
    else:
      print('Not Found: rclone.conf [file]')

  # Check 'Service Accounts' file/folder

  if Path(IMPORT_ACCOUNTS_ZIP).exists() and Path(IMPORT_ACCOUNTS_FOLDER).exists():

    choice = input('Found both! "accounts.zip" [file] & "accounts" [folder]\n Select one: 0 = accounts.zip [file] or 1 = accounts [folder] or 2 = Ignore\n')

    if choice not in ['0','1','2']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif choice == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
    elif choice == '0':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_ZIP).exists():

    is_import = input('Found: accounts.zip [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_FOLDER).exists():

    is_import = input('Found: accounts [folder] | Do you want to import ? 0=No & 1=Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
  else:
    print('Not Found: accounts.zip [file] or accounts [folder]')

  # Check 'mega.json' file

  if Path(IMPORT_MEGA_JSON).exists():

    is_import = input('Found: mega.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_MEGA_JSON" "$IMPORT_TO"
      print('Successfully imported "mega.json" file! (1)')

  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(IMPORT_GOFILE_JSON).exists():

    is_import = input('Found: gofile.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_GOFILE_JSON" "$IMPORT_TO"
      print('Successfully imported "gofile.json" file! (1)')

  else:
    print('Not Found: gofile.json [file]')

  # Check 'telegram.json' file

  if Path(IMPORT_TELEGRAM_JSON).exists():

    is_import = input('Found: telegram.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_TELEGRAM_JSON" "$IMPORT_TO"
      print('Successfully imported "telegram.json" file! (1)')

  else:
    print('Not Found: telegram.json [file]')

  # Check '*.session' files

  if len(IMPORT_SESSION_FILES) != 0:

    is_import = input('Found: Telegram session files | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      for IMPORT_SESSION_FILE in IMPORT_SESSION_FILES:
        !rclone copy "$IMPORT_SESSION_FILE" "$IMPORT_TO"
      print("Successfully imported telegram session files! (1)")

  else:
    print('Not Found: .session [file]')

# Restore Configuration(s) from MCT-Config folder

if not Path(DRIVE_CONFIG_DIR).exists():
  end('Not Found: MCT-Config [folder]')
else:
  print('Found: MCT-Config [folder]')

# Paths
ROOT = "/content/"
DRIVE_RCLONE_CONF = f"{CMP}/MyDrive/MCT-Config/rclone.conf"
ROOT_RCLONE_CONF = "/root/.config/rclone/"
DRIVE_ACCOUNTS_ZIP = f"{CMP}/MyDrive/MCT-Config/accounts.zip"
DRIVE_MEGA_JSON = f"{CMP}/MyDrive/MCT-Config/mega.json"
DRIVE_GOFILE_JSON = f"{CMP}/MyDrive/MCT-Config/gofile.json"
DRIVE_TELEGRAM_JSON = f"{CMP}/MyDrive/MCT-Config/telegram.json"
DRIVE_SESSION_FILES = list(Path(f"{CMP}/MyDrive/MCT-Config/").glob("*.session"))

# Check 'rclone.conf' file

if Path(DRIVE_RCLONE_CONF).exists():
  !rclone copy "$DRIVE_RCLONE_CONF" "$ROOT_RCLONE_CONF"
  print('Restored: rclone.conf [file]')
else:
  print('Not Found: rclone.conf [file]')

# Check 'Service Accounts' file

if Path(DRIVE_ACCOUNTS_ZIP).exists():
  !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$ROOT"
  print('Restored: accounts.zip [file]')
else:
  print('Not Found: accounts.zip [file]')

# Check 'mega.json' file

if Path(DRIVE_MEGA_JSON).exists():
  !rclone copy "$DRIVE_MEGA_JSON" "$ROOT"
  print('Restored: mega.json [file]')
else:
  print('Not Found: mega.json [file]')

# Check 'gofile.json' file

if Path(DRIVE_GOFILE_JSON).exists():
  !rclone copy "$DRIVE_GOFILE_JSON" "$ROOT"
  print('Restored: gofile.json [file]')
else:
  print('Not Found: gofile.json [file]')

# Check 'telegram.json' file

if Path(DRIVE_TELEGRAM_JSON).exists():
  !rclone copy "$DRIVE_TELEGRAM_JSON" "$ROOT"
  print('Restored: telegram.json [file]')
else:
  print('Not Found: telegram.json [file]')

# Check '*.session' files

if len(DRIVE_SESSION_FILES) != 0:
  for DRIVE_SESSION_FILE in DRIVE_SESSION_FILES:
    !rclone copy "$DRIVE_SESSION_FILE" "$ROOT"
  print('Restored: *.session [files]')
else:
  print('Not Found: *.session [files]')

if CUSTOM_EXPORT_PATH:

  #Paths
  EXPORT_TO = f"{Custom_Path}/MCT-Config/"

  Custom_Path = input('Enter the directory path to where configuration file(s) needs to be exported:\n')

  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Path not found!")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Path is not a valid directory.")
  elif not Path(DRIVE_CONFIG_DIR):
    !rclone mkdir "$DRIVE_CONFIG_DIR"

  # Check 'rclone.conf' file

  if Path(DRIVE_RCLONE_CONF).exists():

    is_export = input('Found: rclone.conf [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_RCLONE_CONF" "$EXPORT_TO"
      print('Export: rclone.conf [file] (1)')
  else:
    print('Not Found: rclone.conf [file]')

    # Check 'Service Accounts' file

  if Path(DRIVE_ACCOUNTS_ZIP).exists():

    is_export = input('Found accounts.zip [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$EXPORT_TO"
      print('Export: accounts.zip [file] (1)')
  else:
    print('Not Found: accounts.zip [file]')

  # Check 'mega.json' file

  if Path(DRIVE_MEGA_JSON).exists():

    is_export = input('Found mega.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_MEGA_JSON" "$EXPORT_TO"
      print('Export: mega.json [file] (1)')
  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(DRIVE_GOFILE_JSON).exists():

    is_export = input('Found gofile.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0', '1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_GOFILE_JSON" "$EXPORT_TO"
      print('Export: gofile.json [file] (1)')
  else:
    print('Not Found: gofile.json [file]')

  # Check 'telegram.json' file

  if Path(DRIVE_TELEGRAM_JSON).exists():

    is_export = input('Found telegram.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_TELEGRAM_JSON" "$EXPORT_TO"
      print('Export: telegram.json [file] (1)')
  else:
    print('Not Found: telegram.json [file]')

  if len(DRIVE_SESSION_FILES) != 0:

    is_export = input('Found Telegram session files | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      for DRIVE_SESSION_FILE in DRIVE_SESSION_FILES:
        !rclone copy "$DRIVE_SESSION_FILE" "$EXPORT_TO"
      print("Export: Telegram session files (1)")
  else:
    print("Not Found: Telegram session [files]")

In [ ]:
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 gradio_client==0.2.7 -U

!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone https://github.com/camenduru/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
!git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
!git clone https://github.com/jexom/sd-webui-depth-lib /content/stable-diffusion-webui/extensions/sd-webui-depth-lib
!git clone https://github.com/hnmr293/posex /content/stable-diffusion-webui/extensions/posex
!git clone https://github.com/nonnonstop/sd-webui-3d-open-pose-editor /content/stable-diffusion-webui/extensions/sd-webui-3d-open-pose-editor
!git clone https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
!git clone https://github.com/etherealxx/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
!git clone https://github.com/camenduru/stable-diffusion-webui-catppuccin /content/stable-diffusion-webui/extensions/stable-diffusion-webui-catppuccin
!git clone https://github.com/bpawnzZ/091sd-webui-faceswaplab.git /content/stable-diffusion-webui/extensions/091sd-webui-faceswaplab
!git clone https://github.com/Bing-su/adetailer.gi  /content/stable-diffusion-webui/extensions/adetailer
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg /content/stable-diffusion-webui/extensions/stable-diffusion-webui-rembg
!git clone https://github.com/ashen-sensored/stable-diffusion-webui-two-shot /content/stable-diffusion-webui/extensions/stable-diffusion-webui-two-shot
!git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper /content/stable-diffusion-webui/extensions/sd-webui-aspect-ratio-helper
!git clone https://github.com/tjm35/asymmetric-tiling-sd-webui /content/stable-diffusion-webui/extensions/asymmetric-tiling-sd-webui
!rclone copy 1d:AI/automatic1111/models/ckpt /content/stable-diffusion-webui/models/Stable-diffusion --s3-upload-concurrency 32  --s3-chunk-size 128000 --fast-list -P
!rclone copy 1d:AI/automatic1111/models/lora /content/stable-diffusion-webui/models/lora --s3-upload-concurrency 32  --s3-chunk-size 128000 --fast-list -P
!rclone copy 1d:AI/automatic1111/models/insightface /content/stable-diffusion-webui/models/insightface --s3-upload-concurrency 32  --s3-chunk-size 128000 --fast-list -P
!rclone copy 1d:AI/automatic1111/models/codeformer /content/stable-diffusion-webui/models/codeformer --s3-upload-concurrency 32  --s3-chunk-size 128000 --fast-list -P
!rclone copy 1d:AI/automatic1111/models/esrgan /content/stable-diffusion-webui/models/ESRGAN --s3-upload-concurrency 32  --s3-chunk-size 128000 --fast-list -P
!rclone copy 1d:AI/automatic1111/models/gfpgan /content/stable-diffusion-webui/models/GFPGAN --s3-upload-concurrency 32  --s3-chunk-size 128000 --fast-list -P
!rclone copy 1d:AI/automatic1111/embeddings /content/stable-diffusion-webui/embeddings --s3-upload-concurrency 32  --s3-chunk-size 128000 --fast-list -P
!rclone copy 1d:AI/automatic1111/models/faceswaplab /content/stable-diffusion-webui/models/faceswaplab --s3-upload-concurrency 32  --s3-chunk-size 128000 --fast-list -P
!rclone copy 1d:AI/automatic1111/models/adetailer /content/stable-diffusion-webui/models/adetailer --s3-upload-concurrency 32  --s3-chunk-size 128000 --fast-list -P

%cd /content/stable-diffusion-webui
!git reset --hard
!git -C /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai reset --hard


!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/rev-animated/resolve/main/revAnimated_v11.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o revAnimated_v11.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o revAnimated_v11.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/rev-animated/resolve/main/revAnimated_v11-inpainting.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o revAnimated_v11-inpainting.safetensors

!sed -i -e '''/from modules import launch_utils/a\import os''' /content/stable-diffusion-webui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/stable-diffusion-webui/modules/shared.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

In [ ]:
!pip install git+https://github.com/demotomohiro/remocolab.git
import remocolab
remocolab.setupSSHD()